In [12]:
list_a = ['a', 'b', 'c']
list_b = [1, 2, 3]
for x,y in zip(list_a,list_b):
    print(x,y)


a 1
b 2
c 3


In [ ]:
# 개와 고양이 binary classification 예제의 TFRecord 버전

import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tqdm.notebook import tqdm
from PIL import Image
import io

############ Tensorflow에서 제공된 Type별 Feature 생성 코드 ############

def _bytes_feature(value):
    # string / byte 타입을 받아서 byte list를 리턴.
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    # float / double 타입을 받아서 float list를 리턴
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _floatarray_feature(array):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=array))

def _int64_feature(value):
    # bool / enum / int / uint 타입을 받아서 int64 list를 리턴
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


def _validate_text(text):
    """If text is not str or unicode, then try to convert it to str."""
    if isinstance(text, str):
        return text
    elif isinstance(text, 'unicode'):
        return text.encode('utf8', 'ignore')
    else:
        return str(text)

# DataFrame을 하나 생성해요!
# DataFrame의 column은 2개를 사용해요. (경로포함한 filename, label)

src = '/content/drive/MyDrive/Machine Learning Colab/CAT_DOG/cat_dog_tfrecord/'

df = pd.DataFrame(os.listdir(src),
                  columns=['filename'])

df['label'] = ~df['filename'].str.contains('cat')

df = df.astype({'label' : 'int'})

df['filename'] = df['filename'].map(lambda x : src + x)

display(df)

### TFRecord 생성 함수 ###

# id_list : 이미지 파일명을 가지고 있는 list
# label_list : 이미지 파일의 label을 가지고 있는 list
# tfrecords_name : tfrecord 파일의 이름(train, validation용도로 파일을 따로 만들어야 하기 때문)

def to_tfrecords(id_list, label_list, tfrecords_name):
    print("converting 시작!!")

    options = tf.io.TFRecordOptions(compression_type='GZIP')

    with tf.io.TFRecordWriter(path=os.path.join(tfrecords_name + '.tfrecords'),
                              options=options) as writer:

        for id_, label_ in tqdm(zip(id_list,label_list),
                                total=len(id_list),
                                position=0,
                                leave=True):
            image_path = id_  # 이미지의 경로
            _binary_image = tf.io.read_file(image_path)  # 이미지 파일로부터 binary 데이터를 추출

            my_features = tf.train.Features(feature={
                'image_raw': _bytes_feature(_binary_image),
                'label': _int64_feature(label_),
                'id': _bytes_feature(id_.encode())
            })

            string_set = tf.train.Example(features=my_features)

            writer.write(string_set.SerializeToString())

train_ids, val_ids, train_label, val_label = \
train_test_split(df['filename'],
                 df['label'],
                 test_size=0.2,
                 random_state=0)

to_tfrecords(train_ids,train_label,'cat_dog_train')

,filename,label
0,/content/drive/MyDrive/Machine Learning Colab/...,0
1,/content/drive/MyDrive/Machine Learning Colab/...,0
2,/content/drive/MyDrive/Machine Learning Colab/...,0
3,/content/drive/MyDrive/Machine Learning Colab/...,0
4,/content/drive/MyDrive/Machine Learning Colab/...,0
...,...,...
3995,/content/drive/MyDrive/Machine Learning Colab/...,1
3996,/content/drive/MyDrive/Machine Learning Colab/...,1
3997,/content/drive/MyDrive/Machine Learning Colab/...,1
3998,/content/drive/MyDrive/Machine Learning Colab/...,1


converting 시작!!
